In [2]:
# Define the URL endpoints for the API
BASE_URL = "https://fantasy.premierleague.com/api/"
endpoints = {
    "main": "bootstrap-static/",
    "player_stats": "element-summary/",
    "manager_info": "entry/",
    "manager_history": "history/",
    "leagues": "leagues-classic/"
}

my_team_id = 633740
league_id = 80477

In [3]:
import requests
import pandas as pd
import sqlite3

# Collect the main underlying fpl information
def get_fpl_data(base_url:str = BASE_URL, endpoints:dict = endpoints) -> dict | None:
    url = f"{base_url}{endpoints['main']}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        
        # Create DataFrames for each main element
        dataframes = {
            'chips': pd.DataFrame(data['chips']), # gamechips
            'months': pd.DataFrame(data['phases']),
            'football_players': pd.DataFrame(data['elements']),
            'football_teams': pd.DataFrame(data['teams']),
            'game_weeks': pd.DataFrame(data['events']),  # gameweeks
            'element_types': pd.DataFrame(data['element_types']),  # player positions
            'element_stats': pd.DataFrame(data['element_stats'])
        }
        
        return dataframes
    else:
        print(f"Failed to retrieve FPL data. Status code: {response.status_code}")
        return None

def save_initial_info_to_database(dataframes: dict, db_name: str = "fpl_data.db", verbose: bool = False) -> None:
    try:
        conn = sqlite3.connect(f"./data/{db_name}")

        for table_name, df in dataframes.items():
            if verbose:
                print(f"Processing table: {table_name}")
            
            for column in df.columns:
                if df[column].dtype == 'object':
                    # Check if column has any non-null values
                    non_null_values = df[column].dropna()
                    if len(non_null_values) > 0:
                        first_value = non_null_values.iloc[0]
                        
                        if isinstance(first_value, (dict, list)):
                            if verbose:
                                print(f"Converting column {column} from {type(first_value)} to string")
                            df[column] = df[column].apply(lambda x: str(x) if x is not None else None)
            
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            if verbose:
                print(f"Successfully saved table: {table_name}")

        conn.commit()
    except Exception as e:
        print(f"Error saving to database: {str(e)}")
        raise
    finally:
        conn.close()

def initialise_fpl_database() -> None:
    dataframes = get_fpl_data()
    if dataframes:
        save_initial_info_to_database(dataframes)

dataframes = get_fpl_data()

In [ ]:
column = 'football_players'

print(dataframes[column].columns)
print(dataframes[column].head(10))
preview = dataframes[column]

In [ ]:
def get_manager_gameweek_data(manager_id: int, event_id: int, base_url:str = BASE_URL, endpoints:dict = endpoints) -> dict | None:
    url = f"{base_url}{endpoints['manager_info']}{manager_id}/event/{event_id}/picks/"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Create DataFrames for each main element
        gameweek_data = {}
        
        # Dictionary mapping of keys and their data sources
        data_mapping = {
            'active_chips': 'active_chip',
            'automatic_subs': 'automatic_subs',
            'event_history': 'entry_history',
            'player_picks': 'picks'
        }

        # Create DataFrames only for existing, non-null data
        for df_key, data_key in data_mapping.items():
            try:
                if data_key in data and data[data_key] is not None:
                    # Handle single-row data differently
                    if df_key == 'event_history':
                        gameweek_data[df_key] = pd.DataFrame([data[data_key]])
                    elif isinstance(data[data_key], (list, dict)):
                        # If it's a list, create DataFrame directly
                        if isinstance(data[data_key], list):
                            gameweek_data[df_key] = pd.DataFrame(data[data_key])
                        # If it's a dict, wrap it in a list
                        else:
                            gameweek_data[df_key] = pd.DataFrame([data[data_key]])
                    else:
                        # For scalar values, create a single-cell DataFrame
                        gameweek_data[df_key] = pd.DataFrame([[data[data_key]]])
                else:
                    gameweek_data[df_key] = None
            except Exception as e:
                print(f"Error processing {df_key}: {str(e)}")
                gameweek_data[df_key] = None

        return gameweek_data
    
    else:
        print(f"Failed to retrieve player gameweek data. Status code: {response.status_code}")
        return None

In [ ]:
def get_league_standings(league_id: int, base_url:str = BASE_URL, endpoints:dict = endpoints) -> dict| None:
    url = f"{base_url}{endpoints['leagues']}{league_id}/standings/"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        league_data = {
            'league_info': pd.DataFrame([data['league']]), # league information
            'standings': pd.DataFrame(data['standings']['results'])  # league standings
        }

        return league_data
    else:
        print(f"Failed to retrieve league standings. Status code: {response.status_code}")
        return None
    
league_data = get_league_standings(league_id)
league_info = league_data['league_info']
standings = league_data['standings'].head(10)


In [ ]:
def get_manager_history(manager_id: int, base_url:str = BASE_URL, endpoints:dict = endpoints) -> dict | None:
    url = f"{base_url}{endpoints['manager_history']}{manager_id}/{endpoints['manager_history']}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        manager_history = {
            'current_season': pd.DataFrame(data['history']), # current season performance
            'past_seasons': pd.DataFrame(data['history_past']), # past seasons performance
            'chips': pd.DataFrame(data['chips']) # chips used
        }

        return manager_history
    else:
        print(f"Failed to retrieve manager history. Status code: {response.status_code}")
        return None

In [ ]:
current_week = fpl_data['game_weeks'][fpl_data['game_weeks']['is_current']].iloc[0]['id']

In [ ]:
def collect_manager_data_from_league(league_id: int, main_fpl_data: dict) -> pd.DataFrame | None:
    league_data = get_league_standings(league_id)
    if league_data is None:
        print("No league data found.")
        return None
    
    current_week = main_fpl_data['game_weeks'][main_fpl_data['game_weeks']['is_current']].iloc[0]['id']

    league_manager_data = []

    for manager in league_data['standings']['entry']:
        week = 1
        while week <= current_week:
            manager_gameweek_data = get_manager_gameweek_data(manager, week)

            if manager_gameweek_data is not None:
                event_history = manager_gameweek_data['event_history']
                event_history['manager_id'] = manager

                league_manager_data.append(event_history)
            week += 1

    return pd.concat(league_manager_data, ignore_index=True) if league_manager_data else None

league_fg = collect_manager_data_from_league(league_id, fpl_data)

In [59]:
import time
player_ids = dataframes['football_players']['id'].tolist()[:10]
total_players = len(player_ids)
current_player = 1

for pid in player_ids:
    url = f"https://fantasy.premierleague.com/api/element-summary/{pid}"
    response = requests.get(url)

    player_stats = []

    if response.status_code == 200:
        data = response.json()

        player_name = dataframes['football_players'][dataframes['football_players']['id'] == pid]['web_name'].values[0]

        print(f"Getting data {current_player} of {total_players}, ({player_name})")

        player_data = pd.DataFrame(data['history'])

        player_stats.append(player_data)

        print(f"Data collected for {player_name}. Sleeping for 2 seconds to avoid rate limiting...")


    all_player_stats = pd.concat(player_stats, ignore_index=True) if player_stats else None
    current_player += 1

    time.sleep(2)

Getting data 1 of 10, (Raya)
Data collected for Raya. Sleeping for 2 seconds to avoid rate limiting...
Getting data 2 of 10, (Arrizabalaga)
Data collected for Arrizabalaga. Sleeping for 2 seconds to avoid rate limiting...
Getting data 3 of 10, (Hein)
Data collected for Hein. Sleeping for 2 seconds to avoid rate limiting...
Getting data 4 of 10, (Setford)
Data collected for Setford. Sleeping for 2 seconds to avoid rate limiting...
Getting data 5 of 10, (Gabriel)
Data collected for Gabriel. Sleeping for 2 seconds to avoid rate limiting...
Getting data 6 of 10, (Saliba)
Data collected for Saliba. Sleeping for 2 seconds to avoid rate limiting...
Getting data 7 of 10, (Calafiori)
Data collected for Calafiori. Sleeping for 2 seconds to avoid rate limiting...
Getting data 8 of 10, (J.Timber)
Data collected for J.Timber. Sleeping for 2 seconds to avoid rate limiting...
Getting data 9 of 10, (Kiwior)
Data collected for Kiwior. Sleeping for 2 seconds to avoid rate limiting...
Getting data 10 of 

In [1]:
import requests
import pandas as pd
import sqlite3

url = f"https://fantasy.premierleague.com/api/element-summary/1"
response = requests.get(url)
data = response.json()
player_data = pd.DataFrame(data['history'])

player_stats = []

player_stats.append(player_data)
all_player_stats = pd.concat(player_stats, ignore_index=True)

In [ ]:
url = f"https://fantasy.premierleague.com/api/element-summary/2"
response = requests.get(url)
data = response.json()
player_data = pd.DataFrame(data['history'])

player_stats.append(player_data)
all_player_stats = pd.concat(player_stats, ignore_index=True)

In [6]:
#player_name = dataframes['football_players'][dataframes['football_players']['id'] == pid]['web_name'].values[0]

data_types = all_player_stats.dtypes

In [18]:
import duckdb
import os 
from dotenv import load_dotenv

load_dotenv()

MOTHERDUCK_TOKEN = os.environ.get("MOTHERDUCK_TOKEN")

con = duckdb.connect(f"md:?motherduck_token={MOTHERDUCK_TOKEN}")

con.sql("SHOW DATABASES").show()

┌───────────────────────┐
│     database_name     │
│        varchar        │
├───────────────────────┤
│ fpl                   │
│ last_fm               │
│ md_information_schema │
│ postcodes             │
└───────────────────────┘



In [33]:
import duckdb

con = duckdb.connect(database='data/fpl_data.duckdb', read_only = False)

In [32]:
con.close()

In [ ]:
import time
url = "https://fantasy.premierleague.com/api/element-summary"
player_ids = dataframes['football_players']['id'].tolist()
total_players = len(player_ids)
current_player = 1
player_stats = []
all_player_stats = pd.DataFrame()
row_id = 1

for pid in player_ids:
    response = requests.get(f"{url}/{pid}")
    if response.status_code == 200:
        data = response.json()
        player_name = dataframes['football_players'][dataframes['football_players']['id'] == pid]['web_name'].values[0]

        print(f"Getting data {current_player} of {total_players}, ({player_name})")

        player_data['id'] = row_id
        player_data = pd.DataFrame(data['history'])

        player_stats.append(player_data)
        all_player_stats = pd.concat([all_player_stats,player_data], ignore_index=True)
        print(f"Data collected for {player_name}. Sleeping for 2 seconds to avoid rate limiting...")

        row_id += 1
        current_player += 1

        time.sleep(3)

In [21]:
duckdb.sql("CREATE TABLE IF NOT EXISTS all_player_stats AS SELECT * FROM all_player_stats")

duckdb.sql("INSERT INTO all_player_stats SELECT * FROM all_player_stats")

duckdb.sql("SELECT * FROM all_player_stats").show()

┌─────────┬─────────┬───────────────┬──────────────┬──────────┬──────────────────────┬──────────────┬──────────────┬───────┬──────────┬─────────┬──────────────┬─────────┬──────────────┬────────────────┬───────────┬─────────────────┬──────────────────┬──────────────┬───────────┬───────┬───────┬───────┬───────────┬────────────┬─────────┬───────────┬─────────────────────────────────┬────────────┬─────────┬────────────────────────┬────────┬────────────────┬──────────────────┬────────────────────────────┬─────────────────────────┬───────┬───────────────────┬──────────┬──────────────┬───────────────┐
│ element │ fixture │ opponent_team │ total_points │ was_home │     kickoff_time     │ team_h_score │ team_a_score │ round │ modified │ minutes │ goals_scored │ assists │ clean_sheets │ goals_conceded │ own_goals │ penalties_saved │ penalties_missed │ yellow_cards │ red_cards │ saves │ bonus │  bps  │ influence │ creativity │ threat  │ ict_index │ clearances_blocks_interceptions │ recoveries │ t

In [39]:
for key, value in dataframes.items():
    print(f"Processing table: {key}")
    
    duckdb.sql(f"CREATE TABLE IF NOT EXISTS {key} AS SELECT * FROM value")
    duckdb.sql(f"INSERT INTO {key} SELECT * FROM value")

Processing table: chips
Processing table: months
Processing table: football_players
Processing table: football_teams
Processing table: game_weeks
Processing table: element_types
Processing table: element_stats
